# H20.Ai

In [1]:
from boto.s3.connection import Location
from boto.s3.key import Key
from configparser import ConfigParser
from sklearn import metrics
from sklearn import svm
from sklearn.cross_validation  import cross_val_score
from sklearn.cross_validation import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix
from sklearn.metrics import roc_curve
from sklearn.neural_network import MLPClassifier
from statsmodels.formula.api import logit, probit, poisson, ols
import boto
import datetime
import glob
import logging
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
import pickle
import shutil
import statsmodels.api as sm
import time
import zipfile
import requests

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
    config = ConfigParser()

    config_file = os.path.join(os.path.dirname('__file__'), 'config.ini')
    config.read(config_file)
    default = config['user.data']
    user=default['username']
    passwd=default['password']
    trainQ=default['trainQtr']
    testQ=default['testQtr']

    print("USERNAME=" + user)
    print("PASSWORD=" + passwd)
    print("TRAINQUARTER=" + (trainQ))
    print("TESTQUARTER=" + (testQ))

USERNAME=yadav.ank@Husky.neu.edu
PASSWORD=?{BrQ9L\
TRAINQUARTER=Q12005
TESTQUARTER=Q22005


In [3]:
df=pd.read_csv('trainData_' + trainQ + '.csv')
traincols=['MONTHLY_REPORTING_PERIOD','CURRENT_ACTUAL_UPB','LOAN_AGE',
           'REM_MTH_LEGAL_MATURITY','CURRENT_INTEREST_DATE','CURRENT_DEFERRED_UPB']
y_train=df['DELINQUENT']
Train_DF=df[traincols]
Train_DF.head()


,MONTHLY_REPORTING_PERIOD,CURRENT_ACTUAL_UPB,LOAN_AGE,REM_MTH_LEGAL_MATURITY,CURRENT_INTEREST_DATE,CURRENT_DEFERRED_UPB
0,200504,190000.0,0,360,5.625,0.0
1,200505,190000.0,1,359,5.625,0.0
2,200506,190000.0,2,358,5.625,0.0
3,200507,189000.0,3,357,5.625,0.0
4,200508,189000.0,4,356,5.625,0.0


In [4]:
test_df=pd.read_csv('testData_' + testQ + '.csv')
testcols = ['MONTHLY_REPORTING_PERIOD', 'CURRENT_ACTUAL_UPB', 'LOAN_AGE',
                'REM_MTH_LEGAL_MATURITY', 'CURRENT_INTEREST_DATE', 'CURRENT_DEFERRED_UPB']
y_test = test_df['DELINQUENT']
Test_DF = test_df[testcols]
Test_DF.head()

,MONTHLY_REPORTING_PERIOD,CURRENT_ACTUAL_UPB,LOAN_AGE,REM_MTH_LEGAL_MATURITY,CURRENT_INTEREST_DATE,CURRENT_DEFERRED_UPB
0,200507,214000.0,0,360,5.75,0.0
1,200508,214000.0,1,359,5.75,0.0
2,200509,214000.0,2,358,5.75,0.0
3,200510,213000.0,3,357,5.75,0.0
4,200511,213000.0,4,356,5.75,0.0


In [5]:
import h2o

In [6]:
h2o.init(nthreads = -1, max_mem_size = 8)

Checking whether there is an H2O instance running at http://localhost:54321..... not found.
Attempting to start a local H2O server...
; Java HotSpot(TM) 64-Bit Server VM (build 25.171-b11, mixed mode)
  Starting server from C:\ProgramData\Anaconda3\lib\site-packages\h2o\backend\bin\h2o.jar
  Ice root: C:\Users\ankit\AppData\Local\Temp\tmp__tlmyku
  JVM stdout: C:\Users\ankit\AppData\Local\Temp\tmp__tlmyku\h2o_ankit_started_from_python.out
  JVM stderr: C:\Users\ankit\AppData\Local\Temp\tmp__tlmyku\h2o_ankit_started_from_python.err
  Server is running at http://127.0.0.1:54321
Connecting to H2O server at http://127.0.0.1:54321... successful.


H2O cluster uptime:,09 secs
H2O cluster timezone:,America/New_York
H2O data parsing timezone:,UTC
H2O cluster version:,3.22.0.2
H2O cluster version age:,"7 days, 10 hours and 11 minutes"
H2O cluster name:,H2O_from_python_ankit_0d5rf8
H2O cluster total nodes:,1
H2O cluster free memory:,7.111 Gb
H2O cluster total cores:,8
H2O cluster allowed cores:,8
H2O cluster status:,"accepting new members, healthy"


# GLM(Generalized Linear Model)-
In statistics, the generalized linear model (GLM) is a 
flexible generalization of ordinary linear regression that allows for 
response variables that have error distribution models other than a normal distribution.
-Wikipedia

In [7]:
# Import H2O GLM:
from h2o.estimators.glm import H2OGeneralizedLinearEstimator

In [8]:
# Initialize the GLM estimator:
# Similar to R's glm() and H2O's R GLM, H2O's GLM has the "family" argument

glm_fit1 = H2OGeneralizedLinearEstimator(family='binomial', model_id='glm_fit1')

In [17]:
data = h2o.import_file('trainData_' + trainQ + '.csv')  
splits = data.split_frame(ratios=[0.7, 0.15], seed=1)  



Parse progress: |█████████████████████████████████████████████████████████| 100%


In [21]:
train = splits[0]
valid = splits[1]
test = splits[2]
y='DELINQUENT'
x=train.columns
glm_fit1.train(x=x, y=y, training_frame=train)

glm Model Build progress: |███████████████████████████████████████████████| 100%


In [22]:
glm_fit2 = H2OGeneralizedLinearEstimator(family='binomial', model_id='glm_fit2', lambda_search=True)
glm_fit2.train(x=x, y=y, training_frame=train, validation_frame=valid)

glm Model Build progress: |███████████████████████████████████████████████| 100%


In [23]:
glm_perf1 = glm_fit1.model_performance(test)
glm_perf2 = glm_fit2.model_performance(test)

In [24]:
# Print model performance
print(glm_perf1)
print(glm_perf2)


ModelMetricsBinomialGLM: glm
** Reported on test data. **

MSE: 0.043472616801175104
RMSE: 0.20850087961726949
LogLoss: 0.17730706329254492
Null degrees of freedom: 374805
Residual degrees of freedom: 374798
Null deviance: 141863.28887307714
Residual deviance: 132911.50232885117
AIC: 132927.50232885117
AUC: 0.7110495093646431
pr_auc: 0.1102511048021778
Gini: 0.4220990187292861
Confusion Matrix (Act/Pred) for max f1 @ threshold = 0.06565723855250881: 


,0,1,Error,Rate
0,297154.0,60078.0,0.1682,(60078.0/357232.0)
1,10515.0,7059.0,0.5983,(10515.0/17574.0)
Total,307669.0,67137.0,0.1883,(70593.0/374806.0)


Maximum Metrics: Maximum metrics at their respective thresholds



metric,threshold,value,idx
max f1,0.0656572,0.1666608,245.0
max f2,0.0449587,0.2879787,295.0
max f0point5,0.1643609,0.1570266,121.0
max accuracy,0.5337444,0.9531411,5.0
max precision,0.6649307,1.0,0.0
max recall,0.0076718,1.0,399.0
max specificity,0.6649307,1.0,0.0
max absolute_mcc,0.0442826,0.1442982,297.0
max min_per_class_accuracy,0.0446257,0.6628542,296.0
max mean_per_class_accuracy,0.0401133,0.6656463,308.0


Gains/Lift Table: Avg response rate:  4.69 %, avg score:  4.70 %



...

,0,1,Error,Rate
0,281230.0,76002.0,0.2128,(76002.0/357232.0)
1,9179.0,8395.0,0.5223,(9179.0/17574.0)
Total,290409.0,84397.0,0.2273,(85181.0/374806.0)


Maximum Metrics: Maximum metrics at their respective thresholds



metric,threshold,value,idx
max f1,0.0580816,0.1646547,256.0
max f2,0.0451732,0.2885036,291.0
max f0point5,0.1573072,0.1482110,110.0
max accuracy,0.5010211,0.9531064,0.0
max precision,0.4587630,0.3333333,2.0
max recall,0.0100116,1.0,399.0
max specificity,0.5010211,0.9999944,0.0
max absolute_mcc,0.0414089,0.1453653,302.0
max min_per_class_accuracy,0.0448015,0.6610886,292.0
max mean_per_class_accuracy,0.0414089,0.6684976,302.0


Gains/Lift Table: Avg response rate:  4.69 %, avg score:  4.70 %



,group,cumulative_data_fraction,lower_threshold,lift,cumulative_lift,response_rate,score,cumulative_response_rate,cumulative_score,capture_rate,cumulative_capture_rate,gain,cumulative_gain
,1,0.0100025,0.1892158,5.0857849,5.0857849,0.2384636,0.2636385,0.2384636,0.2636385,0.0508706,0.0508706,408.5784934,408.5784934
,2,0.0200023,0.1480758,2.8622286,3.9741551,0.1342049,0.1629046,0.1863412,0.2132783,0.0286218,0.0794924,186.2228589,297.4155058
,3,0.0300022,0.1336232,2.0485135,3.3323316,0.0960512,0.1399075,0.1562472,0.1888235,0.0204848,0.0999772,104.8513503,233.2331621
,4,0.0400020,0.1247404,1.8265912,2.9559216,0.0856457,0.1289680,0.1385980,0.1738606,0.0182656,0.1182429,82.6591207,195.5921625
,5,0.0500019,0.1159500,1.7867590,2.7221016,0.0837780,0.1202116,0.1276346,0.1631314,0.0178673,0.1361102,78.6759000,172.2101577
,6,0.1000011,0.0874803,2.0291664,2.3756432,0.0951441,0.1005163,0.1113898,0.1318247,0.1014567,0.2375669,102.9166431,137.5643248
,7,0.1500003,0.0715444,2.0291664,2.2601530,0.0951441,0.0788396,0.1059746,0.1141633,0.1014567,0.3390236,102.9166431,126.0153030
,8,0.2000021,0.0616865,1.8276317,2.1520198,0.0856945,0.0662515,0.1009045,0.1021850,0.0913850,0.4304086,82.7631744,115.2019824
,9,0.3000005,0.0493331,1.6092212,1.9710902,0.0754536,0.0549345,0.0924210,0.0864352,0.1609195,0.5913281,60.9221163,97.1090155
,10,0.4000016,0.0413060,1.2962192,1.8023713,0.0607775,0.0450552,0.0845100,0.0760901,0.1296233,0.7209514,29.6219238,80.2371300


In [25]:
# Retreive test set AUC
print(glm_perf1.auc())
print(glm_perf2.auc())

0.7110495093646431
0.7101840996510942


AUC is an abbrevation for area under the curve. 
It is used in classification analysis in order to determine which of the used models predicts the classes best.

In [26]:
# Compare test AUC to the training AUC and validation AUC
print(glm_fit2.auc(train=True))
print(glm_fit2.auc(valid=True))

0.70993142021362
0.7111490144184555


# H2ORandomForestEstimator

In [27]:
# Import H2O RF:
from h2o.estimators.random_forest import H2ORandomForestEstimator

In [28]:
# Initialize the RF estimator:

rf_fit1 = H2ORandomForestEstimator(model_id='rf_fit1', seed=1)

In [29]:
rf_fit1.train(x=x, y=y, training_frame=train)

drf Model Build progress: |███████████████████████████████████████████████| 100%


In [30]:
rf_fit2 = H2ORandomForestEstimator(model_id='rf_fit2', ntrees=100, seed=1)
rf_fit2.train(x=x, y=y, training_frame=train)

drf Model Build progress: |███████████████████████████████████████████████| 100%


In [31]:
rf_perf1 = rf_fit1.model_performance(test)
rf_perf2 = rf_fit2.model_performance(test)

In [33]:
rf_fit3 = H2ORandomForestEstimator(model_id='rf_fit3', seed=1, nfolds=5)
rf_fit3.train(x=x, y=y, training_frame=data)

drf Model Build progress: |███████████████████████████████████████████████| 100%


In [36]:
print(rf_fit3)

Model Details
H2ORandomForestEstimator :  Distributed Random Forest
Model Key:  rf_fit3


ModelMetricsRegression: drf
** Reported on train data. **

MSE: 0.030670707608415396
RMSE: 0.17513054447587204
MAE: 0.06761615365992983
RMSLE: 0.1196736362430878
Mean Residual Deviance: 0.030670707608415396

ModelMetricsRegression: drf
** Reported on cross-validation data. **

MSE: 0.030636729869103433
RMSE: 0.1750335107032463
MAE: 0.06774455760015016
RMSLE: 0.11942413233690233
Mean Residual Deviance: 0.030636729869103433
Cross-Validation Metrics Summary: 


,mean,sd,cv_1_valid,cv_2_valid,cv_3_valid,cv_4_valid,cv_5_valid
mae,0.0677448,0.0001963,0.0677688,0.0672191,0.0679803,0.0677835,0.0679720
mean_residual_deviance,0.0306368,0.0001575,0.0304867,0.0302762,0.0308796,0.0307539,0.0307876
mse,0.0306368,0.0001575,0.0304867,0.0302762,0.0308796,0.0307539,0.0307876
r2,0.3143205,0.0033854,0.3141874,0.3235387,0.3106700,0.3107308,0.3124759
residual_deviance,0.0306368,0.0001575,0.0304867,0.0302762,0.0308796,0.0307539,0.0307876
rmse,0.1750326,0.0004503,0.1746043,0.1740006,0.175726,0.1753680,0.1754640
rmsle,0.1194234,0.0003348,0.1191399,0.1186367,0.1199418,0.1196806,0.1197179


Scoring History: 


,timestamp,duration,number_of_trees,training_rmse,training_mae,training_deviance
,2018-11-29 18:46:32,24 min 9.202 sec,0.0,nan,nan,nan
,2018-11-29 18:46:36,24 min 13.517 sec,1.0,0.1945543,0.0697704,0.0378514
,2018-11-29 18:46:43,24 min 20.763 sec,3.0,0.1877694,0.0672317,0.0352573
,2018-11-29 18:46:48,24 min 25.035 sec,4.0,0.1863038,0.0677361,0.0347091
,2018-11-29 18:46:53,24 min 30.009 sec,5.0,0.1844818,0.0673164,0.0340335
---,---,---,---,---,---,---
,2018-11-29 18:49:10,26 min 47.843 sec,42.0,0.1752976,0.0676730,0.0307293
,2018-11-29 18:49:18,26 min 55.133 sec,44.0,0.1753454,0.0676990,0.0307460
,2018-11-29 18:49:25,27 min 1.970 sec,46.0,0.1753057,0.0676932,0.0307321
,2018-11-29 18:49:32,27 min 9.254 sec,48.0,0.1752615,0.0676757,0.0307166



See the whole table with table.as_data_frame()
Variable Importances: 


variable,relative_importance,scaled_importance,percentage
LOAN_SEQ_NO,332123.5937500,1.0,0.3006974
CURRENT_ACTUAL_UPB,165192.4843750,0.4973826,0.1495616
REM_MTH_LEGAL_MATURITY,150994.9062500,0.4546347,0.1367075
C1,128143.9921875,0.3858322,0.1160187
CURRENT_INTEREST_DATE,125346.0390625,0.3774078,0.1134855
LOAN_AGE,115915.0234375,0.3490117,0.1049469
MONTHLY_REPORTING_PERIOD,75144.9218750,0.2262559,0.0680346
CURRENT_DEFERRED_UPB,11650.1845703,0.0350779,0.0105478
